In [1]:
import numpy as np # importing numpy for matrix operations 
from scipy import *
from scipy.optimize import minimize
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython import display 
import itertools
from IPython.display import display
import random, operator
#===================================================#
from random import choices, randint, randrange, random, sample, seed 
from collections import namedtuple
from typing import List, Optional, Callable, Tuple
from functools import partial

In [2]:
#seed(0) #Fix seed 

In [3]:
class IterRegistry(type):
    def __iter__(cls):
        return iter(cls._registry)

In [4]:
class Firm:
    #class attributes intialisation, to be updated with the addition of each instance
    _registry = []
    
    def __init__(self, index, capacity, max_number_of_vessel, design_speed, 
                 min_speed, max_speed, main_engine_power, aux_engine_power,
                 fix_cost, fuel_type):
        '''__init__ a method to describe the poperty that all shipping firm have'''
        self._registry.append(self)
        self.index = index
        self.capacity = capacity # Vessel capacity in TEU per firm, 
        self.max_number_of_vessel = max_number_of_vessel # # of vessels per firms
        self.design_speed = design_speed  #design speed of the vessel in knots
        self.min_speed = min_speed #min vessel speed
        self.max_speed = max_speed #max vessel speed         
        self.main_engine_power =  main_engine_power  # PS_m : main engine power in kW
        self.aux_engine_power =  aux_engine_power  # auxiliary engine power [kW] 
        self.fix_cost = fix_cost #Daily cost of vessel (USD/Day) $25,000
        self.fuel_type = fuel_type
        #self.bau_emission = bau_emission
         
   
    # Firm methods 
    def get_market_share(self , market_instance):
        '''return market share of the firm accoding to it s capacity'''
        self.transport_capacity = np.multiply(self.max_number_of_vessel , self.capacity)
        self.market_share= np.divide(self.transport_capacity, market_instance.market_capacity)
        return self.market_share
        
    def get_firm_param(self, market_instance):
        self.psy = np.multiply (self.market_share , market_instance.market_psy)
        return self.psy
    
    def get_firm_demand(self, market_instance):
        self.firm_demand = np.multiply (self.market_share , market_instance.market_demand)
        return self.firm_demand 
    
    
    def get_min_number_of_vessel(self, market_instance, operational_speed):
        self.time_at_sea = np.true_divide(market_instance.distance,  operational_speed)
        self.voyage_time = self.time_at_sea  + market_instance.port_time 
        self.number_of_trips_to_meet_demand = np.true_divide(self.firm_demand[market_instance.period_index], self.capacity)
        self.max_trips_per_vessel = np.true_divide(market_instance.annual_working_time, self.voyage_time)
        self.min_number_of_vessel_to_meet_demand = np.true_divide(self.number_of_trips_to_meet_demand, self.max_trips_per_vessel)
        return self.min_number_of_vessel_to_meet_demand, self.number_of_trips_to_meet_demand, self.time_at_sea 
    
    def get_number_of_vessel(self):
        self.number_of_vessel = np.ceil(self.min_number_of_vessel_to_meet_demand)
        return self.number_of_vessel
        
        
    def get_ship_energy_efficiency(self, market_instance):
        self.main_fuel_parameter = market_instance.SFOC_main * market_instance.eng_load_main * self.main_engine_power * 10**(-6)
        self.ship_energy = np.multiply(self.main_fuel_parameter, np.power(self.design_speed, -3) )      
        return self.ship_energy
        
    def get_main_fuel_cons(self, market_instance, operational_speed):
        self.main_fuel_cons = market_instance.distance * self.ship_energy * operational_speed**2 *self.number_of_trips_to_meet_demand
        return self.main_fuel_cons
        
    def get_aux_fuel_cons(self, market_instance): 
        self.aux_fuel_parameter = market_instance.SFOC_aux * market_instance.eng_load_aux * self.aux_engine_power * 10**(-6)
        self.aux_fuel_cons =  self.aux_fuel_parameter * self.time_at_sea 
        return self.aux_fuel_cons 
        
    def get_fuel_cost(self, market_instance):
        self.aux_fuel_cost = self.aux_fuel_cons  * market_instance.fuel_data['MGO']['price'] 
        self.main_fuel_cost = self.main_fuel_cons * market_instance.fuel_data[self.fuel_type]['price']  
        self.fuel_cost = self.main_fuel_cost + self.aux_fuel_cost
        return self.fuel_cost
        
    
    def get_total_cost(self):
        self.operating_cost = self.fix_cost * self.number_of_vessel #Fixed Cost
        self.total_cost = self.operating_cost + self.fuel_cost
        return self.total_cost 
    
    def get_revenue(self,market_instance):
        self.revenue = self.firm_demand[market_instance.period_index] * market_instance.freight_rate
        return self.revenue
    
    def get_period_profits(self):
        self.period_profit = self.revenue - self.total_cost 
        return self.period_profit
    
    def get_discounted_period_profit(self,market_instance):
        self.discounted_period_profit = np.multiply( np.power( 1+ market_instance.discount_rate, - market_instance.period_index), self.period_profit) 
        return self.discounted_period_profit
    
    def get_firm_carbon_emission(self, market_instance):
        self.carbon_aux_emision_factor = market_instance.fuel_data['MGO']['carbon_factor']
        self_carbon_aux_emissions = self.carbon_aux_emision_factor * self.aux_fuel_cons 

        self.carbon_main_emision_factor =  market_instance.fuel_data[self.fuel_type]['carbon_factor']
        self_carbon_main_emissions =  self.carbon_main_emision_factor * self.main_fuel_cons

        self_carbon_emissions = self_carbon_main_emissions  + self_carbon_aux_emissions
        return self_carbon_emissions
    
    def get_firm_sulfur_emission(self, market_instance):
        self.sulfur_aux_emision_factor = market_instance.fuel_data['MGO']['sulfur_factor']
        self_sulfur_aux_emissions = self.sulfur_aux_emision_factor * self.aux_fuel_cons 

        self.sulfur_main_emision_factor =  market_instance.fuel_data[self.fuel_type]['sulfur_factor']
        self_sulfur_main_emissions =  self.sulfur_main_emision_factor * self.main_fuel_cons

        self_sulfur_emissions = self_sulfur_main_emissions  + self_sulfur_aux_emissions
        return self_sulfur_emissions

In [5]:
class MarketConfig: 
    """Market Configuration and simulation game set up
    .. version: : 0.1
    
    Parameters
    ----------
    
    Attributes
    ----------
    
    """
    #class attributes
    t0 = 2016 #The starting year for evaluation of the pay-offs
    T = 35  # Planning Horizon t = 2016,...,2040 # 100 year scope 
    year = np.arange(2016, 2051)
    distance = 11810 #nautical miles #Notteboom (2006)  
    port_time = 264 #hours ==> 11 days in a year  #Notteboom (2006)
    demand_16_20 = np.array([1303780  for j in range(5)]) #in TEU #placehplder to be calibrated with the chosen route 
    annual_working_time = 6480 #hours per year, assumption   #hours per year, assumption 
    initial_freight_rate = 1800 ## in US$/TEU 
    SFOC_main = 206 #g/kWh, specific daily main engine fuel oil consumption rate
    SFOC_aux = 221 # specific fuel oil consumption of the auxiliary engine [g/kW h], 
    eng_load_main = 0.8 # % 
    eng_load_aux = 0.5 # engine load of the auxiliary engine [\%]


    def __init__(self, number_of_firms, demand_income_elasticity, demand_price_elasticity, 
                 freight_rate, fuel_data, discount_rate ):
        '''__init__ a method to describe the poperty that the shipping market and simulation game has'''
        self.number_of_firms = number_of_firms
        self.demand_income_elasticity = demand_income_elasticity #Constant income elasticity #IMF
        self.demand_price_elasticity =  demand_price_elasticity #Constant own price elasticity#IMF
        self.freight_rate = freight_rate ## in US$/TEU
        self.fuel_data = fuel_data 
        self.discount_rate = discount_rate 
        #self.bau_industry_emission =  bau_industry_emission 
        #self.bau_pollution_stock = bau_pollution_stock
        self.market_capacity = 0 
        self.pollution_stock = 0
        self.industry_damage = 0
        self.global_abatement_benefits = 0
        
    
    def get_market_capacity (self, firm_instance):
        self.market_capacity += firm_instance.max_number_of_vessel * firm_instance.capacity
        return self.market_capacity 
    
    def get_freight_rate_ratio(self): 
        self.beta = np.divide(self.freight_rate, MarketConfig.initial_freight_rate) 
        return self.beta
   
    def get_market_demand(self):
        '''compute market level demand'''
        #---------> 1.Import real GDP growth data & compute GDP ratio : Source IMF@2020 #
        G_df = pd.read_csv('./data/real_growth_rate.csv') #import IMF data
        #-------> 2.Construct GDP projection path 2016-2050 based on projection growth data 
        gdp_growth= G_df.values[:,1:] 
        g =1 +(gdp_growth/100)  
        self.gdp = np.array([100.00 for j in range(MarketConfig.T)])
        for foo in range (1,MarketConfig.T):
            self.gdp[foo] = g[:,foo]* self.gdp[foo-1]
        #print(self.gdp)
        #--------->  "compute GDP ratio based on IMF@2020"
        gdp_ratio = np.array([1.00 for j in range(MarketConfig.T)])
        for moo in range (1,MarketConfig.T):
            sub_g = g[:,0:moo+1]
            #print(sub_g)
            gdp_ratio[moo] = np.prod(sub_g)
        
        #--------->  "compute freight rate ratio"
        self.freight_rate_ratio_multiplied = np.power(self.beta, self.demand_price_elasticity)
        #print(freight_rate_ratio_multiplied)
        
        #---------> "Project Transport Demand (industry demand)"
        loo =np.multiply(np.power(gdp_ratio, self.demand_income_elasticity), self.freight_rate_ratio_multiplied ) #Will need to be updated in case fuel prices are varied over time  
        self.market_demand = np.multiply(loo, MarketConfig.demand_16_20[0])
        #print(Y) # size = 1 dimesion array with size T=35 years
        self.market_psy = np.multiply(np.power(gdp_ratio, self.demand_income_elasticity), MarketConfig.demand_16_20[0] )
        return self.market_demand, self.gdp, self.market_psy
    
    def set_period_index (self, period):
        self.period_index = period
        return self.period_index

        
    def get_industry_emission(self):
        
        pass
    
    def get_pollution_stock(self):
        pass
    
    def get_damage(self):
        pass
    
    def get_global_abatement_benefits(self):
        pass 
    
    def get_firm_abatement_benefits():
        pass

In [6]:
class Coalition:
    def __init__(self, ):
        pass

In [7]:
class Singelton: 
    def __init__(self,):
        pass

In [8]:

#-------------------------------------------------- MAIN ------------------------------------------------------------- #

####################################### 1. Firm & Market specific Parameters ###########################################

                    #===========================>  Firm 1 Attributes <==========================#
index_1 = 1
capacity_1 = 5905
max_number_of_vessel_1= 50
design_speed_1 = 23.3 #design speed of the vessel in knots
min_speed_1 = 8.5
max_speed_1 = 30
main_engine_power_1 = 41186 # main engine power in kW
aux_engine_power_1 = 2433 # auxiliary engine power [kW] 
fix_cost_1 =  6750000  #Daily cost of vessel (USD/Day) $25,000
fuel_type_1 = "HFO"

                    #===========================>  Firm 2 Attributes <==========================#
index_2 = 2
capacity_2 = 6470
max_number_of_vessel_2 = 50
design_speed_2 = 24.7 #design speed of the vessel in knots
min_speed_2 = 8.5
max_speed_2 = 30
main_engine_power_2 = 56273 # main engine power in kW
aux_engine_power_2 =  2433 # auxiliary engine power [kW] 
fix_cost_2 = 6750000 #Daily cost of vessel (USD/Day) $25,000
fuel_type_2 = "HFO"


                    #===========================> Market Attributes <==========================#
firms = 2
income_elasticity = 0.8 #Constant income elasticity #IMF
price_elasticity = -0.7 #Constant own price elasticity#IMF
freight_rate = 1800 ## in US$/TEU
fuel_data = pd.DataFrame(np.array([[422.50, 525.50, 597.00], 
                                [3.114,3.206,3.206],
                                [0.07,0.01,0.002]]),
                         columns=['HFO', 'ULSFO', 'MGO'],
                         index = ['price', 'carbon_factor', 'sulfur_factor'])
discount = 0.02  ##3  to 5 % transport canada ,#to be updated using the Ramsey rule,#pure rate of time preference of 1.5% + growth rate of consumption g * rate of risk conversion , an elasticity value of 2

In [9]:
""" This cell needs to run once throughout the kernel"""
####################################### 2.Simulation Game Initialisation ###########################################
firm_1 = Firm(index_1, capacity_1, max_number_of_vessel_1, design_speed_1, 
                 min_speed_1, max_speed_1, main_engine_power_1, aux_engine_power_1,
                 fix_cost_1, fuel_type_1) 

firm_2 = Firm(index_2, capacity_2, max_number_of_vessel_2, design_speed_2, 
                 min_speed_2, max_speed_2, main_engine_power_2, aux_engine_power_2,
                 fix_cost_2, fuel_type_2) 

game_config = MarketConfig(firms, income_elasticity, price_elasticity, freight_rate, 
                           fuel_data, discount)

####################################### 3.Get market capacity #################################
for  firm_object in Firm._registry:
    market_capacity_sim = game_config.get_market_capacity(firm_object)

In [10]:
## 4.Get freight ratio for passethrough
beta_sim = game_config.get_freight_rate_ratio()

# 5.Get market demand  
market_demand_sim, gdp_sim, market_psy_sim = game_config.get_market_demand()

# Market Demand

In [11]:
print("market demand", market_demand_sim)
#Insert Graph

market demand [1303780.         1378613.27053661 1417081.19385409 1448735.913652
 1397511.98183639 1455352.28446974 1504050.11466377 1549602.07735933
 1594072.22320876 1638552.17226732 1684273.26074193 1731270.12057536
 1779578.35005746 1829234.54078948 1880276.30540081 1932742.30603904
 1986672.28365504 2042107.08810522 2099088.70909364 2157660.30797763
 2217866.2504608  2279752.14019835 2343364.85334008 2408752.57403719
 2475964.83093995 2545052.53471369 2616068.01660168 2689065.06806397
 2764098.98152235 2841226.59224218 2920506.32138292 3001998.22024985
 3085764.01578064 3171867.15730103 3260372.86458528]


# Firm 1:

In [12]:
# 6.Get firm market share  
firm_1_market_share_sim = firm_1.get_market_share(game_config)
print("firm 1 market share", firm_1_market_share_sim)
 
# 7.Get firm passthrough parameter
firm_1_psy_sim = firm_1.get_firm_param(game_config)


# 7.Get firm level demand  
firm_1_demand_sim = firm_1.get_firm_demand(game_config)
print("firm 1 demand", firm_1_demand_sim)
    
# 9.Get ship energy efficiency  
firm_1_ship_energy_sim = firm_1.get_ship_energy_efficiency(game_config)
print("firm 1 ship energy", firm_1_ship_energy_sim)

firm 1 market share 0.4771717171717172
firm 1 demand [ 622126.94141414  657835.26161767  676191.06664311  691295.80364566
  666853.19214092  694452.94867021  717690.17592643  739426.28418641
  760646.18004426  781870.75371625  803687.56401464  826113.13632303
  849164.45713853  872858.98693833  897214.6734054   922249.96502307
  947983.82504913  974435.7458797  1001625.76381397 1029574.47423094
 1058303.04718958 1087833.24346435 1118187.43102814 1149388.60199512
 1181460.39003639 1214427.08828156 1248313.66771983 1283145.79611457
 1318949.85744561 1355752.97189415 1393583.01638514 1432468.64570306
 1472439.31419674 1513525.29808991 1555757.71841423]
firm 1 ship energy 0.0005365856566237424


# Firm 2

In [13]:
# 6.Get firm market share  
firm_2_market_share_sim = firm_2.get_market_share(game_config)
print("firm 2 market share", firm_2_market_share_sim)

# 7.Get firm passthrough parameter
firm_2_psy_sim = firm_2.get_firm_param(game_config)


# 7.Get firm level demand  
firm_2_demand_sim = firm_2.get_firm_demand(game_config)
print("firm 2 demand", firm_2_demand_sim)


# 9.Get ship energy efficiency  
firm_2_ship_energy_sim = firm_2.get_ship_energy_efficiency(game_config)
print("firm 2 ship energy", firm_2_ship_energy_sim)

firm 2 market share 0.5228282828282829
firm 2 demand [ 681653.05858586  720778.00891894  740890.12721099  757440.11000634
  730658.78969547  760899.33579953  786359.93873734  810175.79317292
  833426.0431645   856681.41855108  880585.6967273   905156.98425233
  930413.89291893  956375.55385115  983061.63199541 1010492.34101596
 1038688.45860591 1067671.34222552 1097462.94527966 1128085.83374669
 1159563.20327122 1191918.89673401 1225177.42231195 1259363.97204207
 1294504.44090355 1330625.44643213 1367754.34888185 1405919.27194941
 1445149.12407673 1485473.62034803 1526923.30499777 1569529.57454679
 1613324.7015839  1658341.85921112 1704615.14617105]
firm 2 ship energy 0.0006154126460269385


# Period Index

In [14]:
# 7.Set period index  
period_index_sim = game_config.set_period_index(1)

# Genetic Algorithm :

In [23]:
#====> Iinitialisation
Chromo = List[float] 
Population = List[Chromo]  

#====> Iinitialisation of function object
PopulateFunc = Callable[[], Population]
FitnessFunc = Callable[[Chromo], float] #int to refelct max # of vessel
PrinterFunc = Callable[[Population, int, FitnessFunc], None]
SelectionFunc = Callable[[Population, FitnessFunc], Tuple[Chromo, Chromo]]  #takes a population and a fitness fn to select 2 solutions to be the parents of our next generaation solution
CrossoverFunc = Callable[[Chromo, Chromo], Tuple[Chromo, Chromo]] #takes 2 genomes and returns 2 new genomes 
MutationFunc = Callable[[Chromo], Chromo] #takes 1 genome and sometimes returns a modified one


In [16]:
#Steps of the GA
#========> Define chromosome as speed level
def generate_chromo(lower_bound:float, 
                    upper_bound:float) -> Chromo:
    return np.random.uniform(lower_bound, upper_bound)

#========> Generate population as a list of size =50 of random speed level
def generate_population(size: int, 
                        chromo_lower_bound: float , 
                        chromo_upper_bound: float) -> Population: 
    return [generate_chromo(chromo_lower_bound, chromo_upper_bound) for _ in range(size)]


In [17]:
#========> Define fitness function based on the simulation game
def fitness (chromo: Chromo, 
             firm: Firm, 
             sim_game: MarketConfig, 
             max_number_of_vessel: int) -> float:
    
    min_number_vessels_sim , trips_to_meet_demand_sim, time_at_sea_sim  = firm.get_min_number_of_vessel(sim_game, chromo)
    print("min number of vessel is", min_number_vessels_sim )
    print("number of trips to meet demand", trips_to_meet_demand_sim )
    print("Time spent at sea", time_at_sea_sim )

    number_vessels_sim  = firm.get_number_of_vessel()
    print("number of vessel is", number_vessels_sim)
    
    #Insert Model Constraints Here, Update Vessel speed constraint here 
    if number_vessels_sim > max_number_of_vessel:
        return 0
    else:
        main_fuel_con_sim = firm.get_main_fuel_cons(sim_game, chromo)
        print('main fuel consumption is ', main_fuel_con_sim)
        
        aux_fuel_con_sim = firm.get_aux_fuel_cons(sim_game)
        print('aux fuel consumption is ', aux_fuel_con_sim)

        fuel_cost_sim = firm.get_fuel_cost(sim_game)
        print( "fuel cost",fuel_cost_sim)
        
        total_cost_sim = firm.get_total_cost()
        print("total cost",total_cost_sim)
        
        revenues_sim = firm.get_revenue(sim_game)
        print("revenues", revenues_sim)
        
        period_profits_sim = firm.get_period_profits()
        print ("period profits ", period_profits_sim)
        
        discounted_period_profits_sim = firm.get_discounted_period_profit(sim_game)
        print("discounted period profits ", discounted_period_profits_sim)
        
        #We will see about the carbon and sulfir emississons 
        carbon_emissions_sim = firm.get_firm_carbon_emission(sim_game)
        print("carbon emissions",carbon_emissions_sim)
        
        sulfur_emissions_sim = firm.get_firm_sulfur_emission(sim_game)
        print( "sulfur emissions", sulfur_emissions_sim)
        
        return  discounted_period_profits_sim

In [18]:
#========> #Survival of the fittest  and parent selection 
def selection_pair(population: Population, fitness_func: FitnessFunc) -> Population:
    '''select a pair of chromo which will be the parent of 2 new chromo for the next generation, Solution with higher fitness should be more likely to be chosen for reproduction'''
    return choices(
        population = population,
        weights=[fitness_func(chromo) for chromo in population],
        k=2
        ) 

In [19]:
#========> #crossover function  
def blend_crossover (a: Chromo, b: Chromo) -> Tuple[Chromo, Chromo]:
    lower = min(a,b) -  0.5 * (max(a,b) - min (a,b))
    upper = max(a,b) +  0.5 * (max(a,b) - min (a,b))                    
    return np.random.uniform (lower, upper, 2) 

#==================================================================#
#Given parents  a : chromo1 and b:chromo2 :
#randomly select a child between
# min(a,b) - alpha *[max(a,b) - min (a,b)] &  max(a,b) + alpha *[max(a,b) - min (a,b)]
# best practice alpha = 0.5  
# def linear_crossover (a: Chromo, b: Chromo) -> Tuple[Chromo, Chromo]:
#     c =  0.5 * a + 0.5 * b
#     d = 1.5 * a - 0.5 * b
#     e = -0.5 * a - 1.5 * b
#     children = [a,b,c,d,e]
#     return  max(fitness_func(crossed) for crossed in children)

# #Given parents  a : chromo1 and b:chromo2 :
# #create 3 solutions: 
# # 0.5 a + 0.5 b
# # 1.5 a - 0.5 b
# #-0.5 a - 1.5 b
# # ====> Children = the 2 best solutions among the 5 

In [26]:
#========> #mutation function: randomly perturb the values
def mutation(chromo: Chromo, mutate_params: tuple, num: int = 1, probability: float = 0.5) -> Chromo: 
    for _ in range(num):
        # if random retuns a value lower or equal to probablity, the chromo falls whitihinn the mutation probabality 
        if random() <= probability:
            chromo += np.random.normal(0, mutate_params['rate'], mutate_params['dim'])
    
        if chromo < mutate_params['lower_bound']:
            chromo = mutate_params['lower_bound']
            
        elif chromo > mutate_params['upper_bound']:
            chromo = mutate_params['upper_bound']
        return chromo

    
 #Normal distribution : A gaussian with mean 0 and and a chosen standard deviation   x_new = xi +N(0, sigma) 
#1. Random mutation 
#2. Non-Uniform Mutation
#3.Normally distributed mutation 
# loc= 0.0
# scale= rate = 0.25  (standard deviation)
# size= dim =1 #output shape


    
     
# Uniform variation: x_new = xi + (ri - 0.5) delta_i
             
#     def mutate(self, mutate_params):
#         self.value += np.random.normal(0, mutate_params['rate'], mutate_params['dim'])
#         for i in range(len(self.value)):
#             if self.value[i] < mutate_params['lower_bound']:
#                 self.value[i] = mutate_params['lower_bound']
#             elif self.value[i] > mutate_params['upper_bound']:
#                 self.value[i] = mutate_params['upper_bound']
            

    



# Main GA call function 
generations = 100 populations = 100 (50 * solution)
The algorithm we will follow is simple. 
Given a `fitness function`, $f(\cdot)$ we are trying to maximize, and for a certain number of generations limit, do the following:
1. Initialize the population with random vessel speed in $V_{min}, V_{max}$; `parents`

2. Compute the discounted profits in the selected period for each chromosome in the population, then use this as the fitness function we are trying to minimize

3. Sample the `parents` with sampling weights based on fitness (i.e. those with greater fitness have greater likelihood of being resampled); these will be used to create `children` for the next generation

4. With some `crossover rate` select pairs of `parents` and randomly combine characteristics

5. With some `mutation rate` mutate the `parents`

6. The population from steps 4-5 become the `children` from the previous generation. Repeate steps 1-7.


In [34]:
#-------------------------------- MAIN GA Function ----------------------------------------- #
def run_evolution(
    populate_func: PopulateFunc, 
    fitness_func: FitnessFunc,
    mutation_func: MutationFunc, 
    generation_limit: int = 100,
    selection_func: SelectionFunc = selection_pair,
    crossover_func: CrossoverFunc = blend_crossover,
    printer: Optional[PrinterFunc] = None) -> Tuple[Population, int]:
    
    #1. Genrate the 1st generation: Initial generation/population
    population = populate_func()

    
    #2 Simulate until you reach generation limit,
    i = 0
    for i in range(generation_limit):
        #print all solutions in the population 
        print("chromos in the population ", population )
        
        #print ("pop", population[0:2]) #print the first 2 solution of the population 
        
        #Step 1 : Sort solutions in the population of generation i based on fitness
        population = sorted(population, 
                            key=lambda chromo: fitness_func(chromo), 
                            reverse=True) 
        if printer is not None:
            printer(population, i, fitness_func)
            
        #Step 2: Implement elitism and pick the top 2 chromo(speed) in the population based on fitness for the next gernation 
        next_generation = population[0:2]  
        print("next generation", next_generation)
        
        
        
        #Step 3: j in range 24 
        for j in range(int(len(population) / 2) - 1):
            #Step 1: Selection 
            parents = selection_func(population, fitness_func)
            print("parent are", parents) 
            #Step 2: Crossover
            offspring_a, offspring_b = crossover_func(parents[0], parents[1])
            print("offspring a ", offspring_a) 
            print("offspring b", offspring_b) 

            #Step 3: mutation 
            offspring_a = mutation_func(offspring_a)
            offspring_b = mutation_func(offspring_b)
            
            #Step 4: Next generation 
            next_generation += [offspring_a, offspring_b]
        
        population = next_generation #update current population with our next generation and start into the next round of the algo by sorting the population and checking if we reached our fitness limit 
    
    #Sort the population one last time in case we run out of generation  
    population = sorted(
            population, 
            key=lambda chromo: fitness_func(chromo), reverse=True)

        
        return population, i

# Runing the GA 

In [35]:
#-------------------------------- MAIN  ----------------------------------------- #
#Update size population to 50 solutions,ie chromo per population
run_evolution( populate_func = partial(generate_population,
                                                    size=50,                                                    
                                                    chromo_lower_bound = min_speed_1 ,
                                                    chromo_upper_bound = max_speed_1),
              fitness_func = partial(fitness,
                                     firm = firm_1,
                                     sim_game = game_config, 
                                     max_number_of_vessel = max_number_of_vessel_1),
              generation_limit = 100,
              
              mutation_func = partial (mutation,
                                       mutate_params = {'lower_bound': min_speed_1, 'upper_bound': max_speed_1, 'rate': 0.25, 'dim': 1}),  
             )


chromos in the population  [26.651405850845755, 28.239831827433765, 13.968870567743217, 23.40895472199404, 20.28195676606663, 23.88558148803189, 15.509809666787962, 29.62942810957704, 14.390681562483103, 14.283571909672762, 24.943292176666564, 22.429963341794135, 24.555716393237308, 11.667414394376573, 20.128356026133787, 19.038017386082025, 22.380446478196117, 18.22198590976985, 17.85659964795432, 23.098570198305488, 25.38909090365114, 15.80073817122846, 29.941725094845633, 21.364950271137737, 14.640847007078833, 19.59220368290065, 20.702437029160095, 26.278330408158286, 12.791660393771416, 16.38730333646262, 20.178690061118765, 10.961397321222115, 26.03996008254728, 10.890064494497926, 26.762972283770154, 11.234606299355292, 12.651639132199758, 21.298309682699248, 11.330059064275623, 13.262249575455817, 9.964125092379373, 28.8611490196506, 20.551701956016608, 27.017490754992743, 16.856719626659427, 8.932724539844596, 16.964496099467233, 22.26754974776015, 22.728926058166522, 24.09561

carbon emissions 844381.4731398114
sulfur emissions 18969.602863873642
min number of vessel is 15.20338813823642
number of trips to meet demand 111.40309256861497
Time spent at sea 620.3377043154633
number of vessel is 16.0
main fuel consumption is  255876.05266028846
aux fuel consumption is  166.7756206232472
fuel cost 108207197.29448396
total cost 216207197.29448396
revenues 1184103470.9118085
period profits  967896273.6173246
discounted period profits  948917915.3111025
carbon emissions 797332.7106238564
sulfur emissions 17911.65723746144
min number of vessel is 14.6256866087735
number of trips to meet demand 111.40309256861497
Time spent at sea 586.7344548489905
number of vessel is 15.0
main fuel consumption is  286024.2234887987
aux fuel consumption is  157.7415046155591
fuel cost 120939406.10227294
total cost 222189406.10227293
revenues 1184103470.9118085
period profits  961914064.8095355
discounted period profits  943053004.7152308
carbon emissions 891185.1512079167
sulfur emiss

discounted period profits  973921580.7498282
carbon emissions 360510.86373717466
sulfur emissions 8086.577397730437
min number of vessel is 18.406392359920858
number of trips to meet demand 111.40309256861497
Time spent at sea 806.6473199460304
number of vessel is 19.0
main fuel consumption is  151327.761874219
aux fuel consumption is  216.86430870187044
fuel cost 64065447.38415255
total cost 192315447.38415253
revenues 1184103470.9118085
period profits  991788023.527656
discounted period profits  972341199.5369176
carbon emissions 471929.91745001613
sulfur emissions 10593.377059812734
min number of vessel is 22.610980040626494
number of trips to meet demand 111.40309256861497
Time spent at sea 1051.2161873155917
number of vessel is 23.0
main fuel consumption is  89104.9682400627
aux fuel consumption is  282.6157927031412
fuel cost 37815570.70967027
total cost 193065570.70967028
revenues 1184103470.9118085
period profits  991037900.2021382
discounted period profits  971605784.5119002
c

number of trips to meet demand 111.40309256861497
Time spent at sea 845.4513156755522
number of vessel is 20.0
main fuel consumption is  137755.45030138423
aux fuel consumption is  227.29662713976734
fuel cost 58337373.83873728
total cost 193337373.83873728
revenues 1184103470.9118085
period profits  990766097.0730712
discounted period profits  971339310.8559521
carbon emissions 429699.1852251206
sulfur emissions 9643.336114351176
min number of vessel is 17.62942892110691
number of trips to meet demand 111.40309256861497
Time spent at sea 761.4535738171838
number of vessel is 18.0
main fuel consumption is  169824.0292281918
aux fuel consumption is  204.7141282332415
fuel cost 71872866.68346627
total cost 193372866.68346626
revenues 1184103470.9118085
period profits  990730604.2283423
discounted period profits  971304513.9493551
carbon emissions 529488.3405117049
sulfur emissions 11888.091474229892
min number of vessel is 22.45872537143335
number of trips to meet demand 111.403092568614

min number of vessel is 12.156838303017958
number of trips to meet demand 111.40309256861497
Time spent at sea 443.1285938946152
number of vessel is 13.0
main fuel consumption is  501448.63078177435
aux fuel consumption is  119.13357151848867
fuel cost 211933169.2474962
total cost 299683169.2474962
revenues 1184103470.9118085
period profits  884420301.6643124
discounted period profits  867078727.1218748
carbon emissions 1561892.9784847335
sulfur emissions 35101.64242186725
min number of vessel is 12.125080443779728
number of trips to meet demand 111.40309256861497
Time spent at sea 441.2813298454869
number of vessel is 13.0
main fuel consumption is  505655.6832512365
aux fuel consumption is  118.63694104430468
fuel cost 213710352.4274509
total cost 301460352.4274509
revenues 1184103470.9118085
period profits  882643118.4843576
discounted period profits  865336390.6709387
carbon emissions 1574992.1476773384
sulfur emissions 35396.135101468644
min number of vessel is 12.053612368047569
n

Time spent at sea 418.2036235969047
number of vessel is 12.0
main fuel consumption is  563002.6205763981
aux fuel consumption is  112.43258049134523
fuel cost 237935729.44408154
total cost 318935729.44408154
revenues 1184103470.9118085
period profits  865167741.467727
discounted period profits  848203668.1056147
carbon emissions 1753550.619327959
sulfur emissions 39410.408305508856
min number of vessel is 11.573554121339996
number of trips to meet demand 111.40309256861497
Time spent at sea 409.2006174791919
number of vessel is 12.0
main fuel consumption is  588048.8955069361
aux fuel consumption is  110.01215380711956
fuel cost 248516335.60750335
total cost 329516335.60750335
revenues 1184103470.9118085
period profits  854587135.3043051
discounted period profits  837830524.8081422
carbon emissions 1831536.9595737047
sulfur emissions 41163.642709793145
min number of vessel is 11.39114175980712
number of trips to meet demand 111.40309256861497
Time spent at sea 398.59021093230905
number

number of vessel is 14.0
main fuel consumption is  364706.9468563025
aux fuel consumption is  139.69323305793378
fuel cost 154172081.90692338
total cost 248672081.90692338
revenues 1184103470.9118085
period profits  935431389.0048851
discounted period profits  917089597.0636128
carbon emissions 1136145.2890157097
sulfur emissions 25529.765666407293
min number of vessel is 13.32860738985437
number of trips to meet demand 111.40309256861497
Time spent at sea 511.2870579697778
number of vessel is 14.0
main fuel consumption is  376665.9825084366
aux fuel consumption is  137.45773603047186
fuel cost 159223439.87822467
total cost 253723439.87822467
revenues 1184103470.9118085
period profits  930380031.0335839
discounted period profits  912137285.3270429
carbon emissions 1173378.5590329852
sulfur emissions 26366.893691062625
min number of vessel is 13.212059322097096
number of trips to meet demand 111.40309256861497
Time spent at sea 504.50778944451696
number of vessel is 14.0
main fuel consu

revenues 1184103470.9118085
period profits  939457199.2778183
discounted period profits  921036469.880214
carbon emissions 1106471.1764063009
sulfur emissions 24862.58707252671
min number of vessel is 13.471559954419128
number of trips to meet demand 111.40309256861497
Time spent at sea 519.6022007276783
number of vessel is 14.0
main fuel consumption is  364706.9468563025
aux fuel consumption is  139.69323305793378
fuel cost 154172081.90692338
total cost 248672081.90692338
revenues 1184103470.9118085
period profits  935431389.0048851
discounted period profits  917089597.0636128
carbon emissions 1136145.2890157097
sulfur emissions 25529.765666407293
min number of vessel is 13.32860738985437
number of trips to meet demand 111.40309256861497
Time spent at sea 511.2870579697778
number of vessel is 14.0
main fuel consumption is  376665.9825084366
aux fuel consumption is  137.45773603047186
fuel cost 159223439.87822467
total cost 253723439.87822467
revenues 1184103470.9118085
period profits 

number of trips to meet demand 111.40309256861497
Time spent at sea 696.1597875206497
number of vessel is 17.0
main fuel consumption is  203174.00079777534
aux fuel consumption is  187.16012231567035
fuel cost 85952749.93008253
total cost 200702749.93008253
revenues 1184103470.9118085
period profits  983400720.9817259
discounted period profits  964118353.9036528
carbon emissions 633283.8738364164
sulfur emissions 14222.554376088907
min number of vessel is 15.908982570600754
number of trips to meet demand 111.40309256861497
Time spent at sea 661.3801189944343
number of vessel is 16.0
main fuel consumption is  225104.26704604665
aux fuel consumption is  177.80973016123716
fuel cost 95212705.23586096
total cost 203212705.23586094
revenues 1184103470.9118085
period profits  980890765.6759475
discounted period profits  961657613.4077916
carbon emissions 701544.7455762861
sulfur emissions 15757.65431268359
min number of vessel is 15.680985175620687
number of trips to meet demand 111.40309256

[13.262249575455817,
 14.283571909672762,
 11.667414394376573,
 12.651639132199758,
 14.390681562483103,
 12.791660393771416,
 14.640847007078833,
 11.234606299355292,
 13.968870567743217,
 15.509809666787962,
 11.330059064275623,
 16.38730333646262,
 15.80073817122846,
 10.890064494497926,
 10.961397321222115,
 9.964125092379373,
 16.856719626659427,
 16.964496099467233,
 17.85659964795432,
 18.22198590976985,
 8.932724539844596,
 19.59220368290065,
 19.038017386082025,
 20.128356026133787,
 20.178690061118765,
 20.28195676606663,
 20.551701956016608,
 20.702437029160095,
 21.298309682699248,
 21.364950271137737,
 22.26754974776015,
 22.380446478196117,
 22.429963341794135,
 22.728926058166522,
 23.098570198305488,
 23.40895472199404,
 24.09561422994871,
 23.88558148803189,
 24.555716393237308,
 24.943292176666564,
 25.38909090365114,
 26.03996008254728,
 26.278330408158286,
 26.651405850845755,
 26.762972283770154,
 27.017490754992743,
 28.239831827433765,
 28.8611490196506,
 29.6294

# Plotting And Analysis: 

**Current Week Objective** : Compute the BAU emissions of sulfur and carbon for each firm Using the GA 

In [ ]:
# 8.Get min # of vessels to meet demand 
firm_1_min_number_vessels_sim, firm_1_trips_to_meet_demand_sim, firm_1_time_at_sea_sim  = firm_1.get_min_number_of_vessel(game_config)
firm_2_min_number_vessels_sim, firm_2_trips_to_meet_demand_sim, firm_2_time_at_sea_sim= firm_2.get_min_number_of_vessel(game_config)


# 9.Get # of vessels to meet demand#
firm_1_number_vessels_sim  = firm_1.get_number_of_vessel()
firm_2_number_vessels_sim  = firm_2.get_number_of_vessel()

# 10.Get main engine fuel consumption #################################
firm_1_main_fuel_con_sim = firm_1.get_main_fuel_cons(game_config)
firm_2_main_fuel_con_sim = firm_2.get_main_fuel_cons(game_config)

# 11.Get Auxi engine fuel consumption  
firm_1_aux_fuel_con_sim = firm_1.get_aux_fuel_cons(game_config)
firm_2_aux_fuel_con_sim = firm_2.get_aux_fuel_cons(game_config)

# 13.Get Firm Cost & Revenues  
firm_1_fuel_cost_sim = firm_1.get_fuel_cost(game_config)
firm_1_total_cost_sim = firm_1.get_total_cost()
firm_1_revenues_sim = firm_1.get_revenue(game_config)
firm_1_period_profits_sim = firm_1.get_period_profits()
firm_1_discounted_period_profits_sim = firm_1.get_discounted_period_profit(game_config)
#===================================================================#
firm_2_fuel_cost_sim = firm_2.get_fuel_cost(game_config)
firm_2_total_cost_sim = firm_2.get_total_cost()
firm_2_revenues_sim = firm_2.get_revenue(game_config)
firm_2_period_profits_sim = firm_2.get_period_profits()
firm_2_discounted_period_profits_sim = firm_2.get_discounted_period_profit(game_config)

# 14.Get Carbon Emissions  
firm_1_carbon_emissions_sim = firm_1.get_firm_carbon_emission(game_config)
firm_2_carbon_emissions_sim = firm_2.get_firm_carbon_emission(game_config)

# 14.Get Sulfur Emissions  
firm_1_sulfur_emissions_sim = firm_1.get_firm_sulfur_emission(game_config)
firm_2_sulfur_emissions_sim = firm_2.get_firm_sulfur_emission(game_config)



## Genetic Algorithm Implementation to solve optimum Vessel Speed and Number of vessels for period index for each firm separately:


## Genetic Algorithm Implementation to solve optimum Vessel Speed and Number of vessels for period index for each firm separately:
1. Create the population of individuals 
Generate an initial population of individuals randomly.

2. Evaluate the fitness of each individual in the population.
The higher the fitness, the longer the individual's properties (DNA) stay  in the population, either by the individual itself or its offspring

3. Repeat as often :
a) Select individuals with a good fitness score for reproduction.
b) Let them produce offsprings.
c) Mutate these offsprings.
d) Evaluate the fitness of each individual in the population.
f) Let the individuals with a bad fitness score die.


4. Pick the individual with the highest fitness as the solution



### a. Individuals


### b.How many individuals are in the first population and how are they generated?
Using uniformly random elements is a good choice becaise we can cover the entire solution space quite well.

### c. Fitness Score
We want to maximize our profit, so we can just use our payoff function

### d.Crosseover


### e. mutate
Example: add Gaussian noise to each offspring. Maybe a mean of zero and a standard deviation of 0.25 is fine. 


### f. population size :
Maintain a constant population size 



### g.  Termination criteria 
until the solutions don’t get better anymore.


# 1. Set up : Hyperparmeters
- How many starting individuals? 100 
- How many parents? 60 (two produce one offspring, like before)
- How many offsprings? 30 
- How many dying individuals? 30  
- How many epochs (cycles) ? 1000

# EnviromentalGovernance class

# 2. Initial population
Individuals = potential solutions to the problem = {vessel speed, number of vessels}

The number of individual (candidate solution) in the population is a hyperparameter. 
Let's start with 10 random individuals with random meaning uniformly drawn numbers between $ V_i^{min}$ and $ V_i^{max} $

and ***number_vessels_sim*** between $ \frac{ψ_{i,t}}{k_t τ} (t_p + \frac{d}{V_i^{max}}) β_t^η  $ and 
$ min(n_i, \frac{ψ_{i,t}}{k_t τ} (t_p + \frac{d}{V_i^{min}}) β_t^η )$

Constraint subroutine elimination: 
- $  $ #Relathonship bw vessel speed and number of vessels

In [ ]:
def _random_init(self, init_params):
    '''intialise individuals through a random permuation '''
    '''must be updated with how to initalise my candidate solutions for vessel speed and fleet size '''
    return np.random.choice(
        a=range(init_params['n_cities']),
        size=init_params['n_cities'],
        replace=False
    )

# 3.Mutation procedure
how can we alter the candidate solutions in a simple way?
* Mutation Methods : Crossover : 

In [ ]:
def mutate(self, mutate_params):
    for _ in range(mutate_params['rate']):
        i, j = np.random.choice(
            a=range(len(self.value)),
            size=2,
            replace=False)
        self.value[i], self.value[j] = self.value[j], self.value[i]

# 4.Crossover Procedure

# 5.Implementation

In [ ]:
class Individual(ABC):
    def __init__(self, value=None, init_params=None):
        if value is not None:
            self.value = value
        else:
            self.value = self._random_init(init_params)

    @abstractmethod
    def pair(self, other, pair_params):
        pass

    @abstractmethod
    def mutate(self, mutate_params):
        pass

    @abstractmethod
    def _random_init(self, init_params):
        pass
    
class TSP(Individual):
    def pair(self, other, pair_params):
        self_head = self.value[:int(len(self.value) * pair_params['alpha'])].copy()
        self_tail = self.value[int(len(self.value) * pair_params['alpha']):].copy()
        other_tail = other.value[int(len(other.value) * pair_params['alpha']):].copy()

        mapping = {other_tail[i]: self_tail[i] for i in range(len(self_tail))}

        for i in range(len(self_head)):
            while self_head[i] in other_tail:
                self_head[i] = mapping[self_head[i]]

        return TSP(np.hstack([self_head, other_tail]))

    def mutate(self, mutate_params):
        for _ in range(mutate_params['rate']):
            i, j = np.random.choice(range(len(self.value)), 2, replace=False)
            self.value[i], self.value[j] = self.value[j], self.value[i]

    def _random_init(self, init_params):
        return np.random.choice(range(init_params['n_cities']), init_params['n_cities'], replace=False)

In [ ]:
class SolutionCandidate(ABC):
    def __init__(self, value=None, init_params=None):
        if value is not None:
            self.value = value
        else:
            self.value = self._random_init(init_params)

    @abstractmethod
    def pair(self, other, pair_params):
        pass

    @abstractmethod
    def mutate(self, mutate_params):
        pass

    @abstractmethod
    def _random_init(self, init_params):
        pass

# Create necessary classes and functions for the genetic algorithm

### 1.Create a fitness function class

In [ ]:
from abc import ABC, abstractmethod

In [ ]:
class Optimization(Individual):
    def pair(self, other, pair_params):
        return Optimization(pair_params['alpha'] * self.value + (1 - pair_params['alpha']) * other.value)

    def mutate(self, mutate_params):
        self.value += np.random.normal(0, mutate_params['rate'], mutate_params['dim'])
        for i in range(len(self.value)):
            if self.value[i] < mutate_params['lower_bound']:
                self.value[i] = mutate_params['lower_bound']
            elif self.value[i] > mutate_params['upper_bound']:
                self.value[i] = mutate_params['upper_bound']

    def _random_init(self, init_params):
        return np.random.uniform(init_params['lower_bound'], init_params['upper_bound'], init_params['dim'])

In [ ]:
class Population:
    def __init__(self, size, fitness, individual_class, init_params):
        self.fitness = fitness
        self.individuals = [individual_class(init_params=init_params) for _ in range(size)]
        self.individuals.sort(key=lambda x: self.fitness(x))

    def replace(self, new_individuals):
        size = len(self.individuals)
        self.individuals.extend(new_individuals)
        self.individuals.sort(key=lambda x: self.fitness(x))
        self.individuals = self.individuals[-size:]

    def get_parents(self, n_offsprings):
        mothers = self.individuals[-2 * n_offsprings::2]
        fathers = self.individuals[-2 * n_offsprings + 1::2]

        return mothers, fathers

In [ ]:
class Evolution:
    def __init__(self, pool_size, fitness, individual_class, n_offsprings, pair_params, mutate_params, init_params):
        self.pair_params = pair_params
        self.mutate_params = mutate_params
        self.pool = Population(pool_size, fitness, individual_class, init_params)
        self.n_offsprings = n_offsprings

    def step(self):
        mothers, fathers = self.pool.get_parents(self.n_offsprings)
        offsprings = []

        for mother, father in zip(mothers, fathers):
            offspring = mother.pair(father, self.pair_params)
            offspring.mutate(self.mutate_params)
            offsprings.append(offspring)
        self.pool.replace(offsprings)

In [ ]:
def fitness(opt):
    return -opt.value[0] * (opt.value[0] - 1) * (opt.value[0] - 2) * (opt.value[0] - 3) * (opt.value[0] - 4)

In [ ]:
from evo import Evolution, Optimization

In [ ]:
evo = Evolution(
    pool_size=10, fitness=fitness, individual_class=Optimization, n_offsprings=3,
    pair_params={'alpha': 0.5},
    mutate_params={'lower_bound': 0, 'upper_bound': 4, 'rate': 0.25, 'dim': 1},
    init_params={'lower_bound': 0, 'upper_bound': 4, 'dim': 1}
                )
n_epochs = 50

for i in range(n_epochs):
    evo.step()

print(evo.pool.individuals[-1].value)

In [ ]:
from evo import Evolution, TSP

def tsp_fitness_creator(cities):
    matrix = []
    for city in cities:
        row = []
        for city_ in cities:
            row.append(np.linalg.norm(city - city_))
        matrix.append(row)
    distances = np.array(matrix)

    def fitness(tsp):
        res = 0
        for i in range(len(tsp.value)):
            res += distances[tsp.value[i], tsp.value[(i + 1) % len(tsp.value)]]
        return -res

    return fitness


def compute_distances(cities):
    distances = []
    for from_city in cities:
        row = []
        for to_city in cities:
            row.append(np.linalg.norm(from_city - to_city))
        distances.append(row)
    return np.array(distances)


def route_length(distances, route):
    length = 0
    for i in range(len(route)):
        length += distances[route[i], route[(i + 1) % len(route)]]
    return length


def plot_route(cities, route, distances):
    length = route_length(distances, route)

    plt.figure(figsize=(12, 8))
    plt.scatter(x=cities[:, 0], y=cities[:, 1], s=1000, zorder=1)
    for i in range(len(cities)):
        plt.text(cities[i][0], cities[i][1], str(i), horizontalalignment='center', verticalalignment='center', size=16,
                 c='white')
    for i in range(len(route)):
        plt.plot([cities[route[i]][0], cities[route[(i + 1) % len(route)]][0]],
                 [cities[route[i]][1], cities[route[(i + 1) % len(route)]][1]], 'k', zorder=0)
    if len(route)>0:
        plt.title(f'Visiting {len(route)} cities in length {length:.2f}', size=16)
    else:
        plt.title(f'{len(cities)} cities', size=16)
    plt.show()


cities = np.array([[35, 51],
                   [113, 213],
                   [82, 280],
                   [322, 340],
                   [256, 352],
                   [160, 24],
                   [322, 145],
                   [12, 349],
                   [282, 20],
                   [241, 8],
                   [398, 153],
                   [182, 305],
                   [153, 257],
                   [275, 190],
                   [242, 75],
                   [19, 229],
                   [303, 352],
                   [39, 309],
                   [383, 79],
                   [226, 343]])

fitness = tsp_fitness_creator(cities)
distances = compute_distances(cities)

evo = Evolution(
    pool_size=100, fitness=fitness, individual_class=TSP, n_offsprings=30,
    pair_params={'alpha': 0.5},
    mutate_params={'rate': 1},
    init_params={'n_cities': 20}
)
n_epochs = 1000

hist = []
for i in range(n_epochs):
    hist.append(evo.pool.fitness(evo.pool.individuals[-1]))
    evo.step()

plt.plot(hist)
plt.show()

plot_route(cities, route=evo.pool.individuals[-1].value, distances=distances)

### 2.Create our initial population

The next step is to define the initial population. Based on the number of weights, each chromosome (solution or individual) in the population will definitely have 6 genes, one gene for each weight. But the question is how many solutions per the population? There is no fixed value for that and we can select the value that fits well with our problem. But we could leave it generic so that it can be changed in the code. Next, we create a variable that holds the number of solutions per population, another to hold the size of the population, and finally, a variable that holds the actual initial population:

### 3.Create the genetic algorithm

#### 3.1. Rank Individuals

#### 3.2. Create a selection function that will be used to make the list of parent routes

#### 3.3. Create mating pool

#### 3.4. Create a crossover function for two parents to create one child

#### 3.5. Create function to run crossover over full mating pool

#### 3.6. Create function to mutate a single route

#### 3.7. Create function to run mutation over entire population

#### 3.8. Put all steps together to create the next generation

#### 3.9. Final step: create the genetic algorithm